# ML Model Management System Architecture

Line of Business.

In [22]:
# standard library
import typing as T
import dataclasses

# third party library
import moto
import pynamodb_mate as pm
import versioned.api as versioned
from s3pathlib import S3Path, context
from boto_session_manager import BotoSesManager
from rich import print as rprint

full list of supported services: https://docs.getmoto.org/en/latest/docs/services/index.html

In [11]:
@dataclasses.dataclass
class AwsMockManager:
    mock_list: list = dataclasses.field()

    @classmethod
    def new(cls, mock_list: list):
        return cls(
            mock_list=[
                mock_service()
                for mock_service in mock_list
            ]
        )

    def mock_start(self):
        for mock_service in self.mock_list:
            mock_service.start()

    def mock_stop(self):
        for mock_service in self.mock_list:
            mock_service.stop()

amm = AwsMockManager.new(
    mock_list = [
        moto.mock_sts,
        moto.mock_s3,
        moto.mock_dynamodb,
        moto.mock_iam,
    ]
)

In [15]:
amm.mock_start()

bsm = BotoSesManager()
print(f"{bsm.principal_arn = }")
print(f"{bsm.aws_account_id = }")

bsm.principal_arn = 'arn:aws:sts::123456789012:user/moto'
bsm.aws_account_id = '123456789012'


In [ ]:
class ModelCatalog(pm.Model):
    class Meta:
        table_name = "model-catalog"
        region = "us-east-1"
        billing_mode = pm.PAY_PER_REQUEST_BILLING_MODE

    namespace = pm.UnicodeAttribute(hash_key=True)

In [ ]:
class ModelRegistry(pm.Model):
    class Meta:
        table_name = "model-registry"
        region = "us-east-1"
        billing_mode = pm.PAY_PER_REQUEST_BILLING_MODE

    url = pm.UnicodeAttribute(hash_key=True)

## Model Registry - Artifacts Store

In [18]:
lob_name = "lob1"
model_name = "model1"

In [24]:
repo = versioned.s3_only_backend.Repository(
    aws_region=bsm.aws_region,
    s3_bucket=f"{bsm.aws_account_id}-{bsm.aws_region}-artifacts",
    s3_prefix=f"ml-models/{lob_name}",
    suffix=".tar.gz",
)
repo.bootstrap(bsm=bsm)
repo.purge_artifact(bsm=bsm, name=model_name)

In [ ]:
artifact = repo.put_artifact(
    bsm=bsm, 
    name=model_name, 
    # content can be any binary data, you just need to create a tar file
    content=f"{lob_name = }, {model_name = }, version = v1".encode("utf8"),
)
rprint(artifact)
print(f"preview on S3: {artifact.s3path.console_url}")
print(f"preview artifact content: {artifact.get_content(bsm=bsm)}")

In [25]:
artifact = repo.put_artifact(
    bsm=bsm, 
    name=model_name, 
    # content can be any binary data, you just need to create a tar file
    content=f"{lob_name = }, {model_name = }, version = v1".encode("utf8"),
)
rprint(artifact)
print(f"preview on S3: {artifact.s3path.console_url}")
print(f"preview artifact content: {artifact.get_content(bsm=bsm)}")

Artifact(
    name='model1',
    version='LATEST',
    update_at='2023-12-21T21:31:46+00:00',
    s3uri='s3://123456789012-us-east-1-artifacts/ml-models/lob1/model1/versions/000000_LATEST.tar.gz',
    sha256='bad4a54d8a97f8dfd50cb79d65b4a2bf35fed453b5d830c295a80927ef2fe5ac'
)

preview on S3: https://console.aws.amazon.com/s3/object/123456789012-us-east-1-artifacts?prefix=ml-models/lob1/model1/versions/000000_LATEST.tar.gz
preview artifact content: b"lob_name = 'lob1', model_name = 'model1', version = v1"
